In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0,'../libs/')
import data_cleaning as dc

In [2]:
df = pd.read_csv("../data/train.csv")
df = dc.classify_colors(df)
df = dc.massage_df(df)

In [3]:
df_dog = df[df.AnimalType=='Dog'].copy(deep=True)
df_cat = df[df.AnimalType=='Cat'].copy(deep=True)

### Catzzzzz

#### fudge data set with desired feature set

In [4]:
dc.fix_cat_breed(df_cat)

In [5]:
cat_features = ['age_numeric', 'neuter_status', 'sex', 'mixed', 'catbreed']
cat_features_dummies = [ 'neuter_status_Intact', 'neuter_status_Neutered', #'age_numeric',
       'neuter_status_Spayed', 'neuter_status_Unknown', 'sex_Female',
       'sex_Male', 'sex_Unknown', 'mixed_Mixed',
       'catbreed_domestic longhair', 'catbreed_domestic mediumhair',
       'catbreed_domestic shorthair', 'catbreed_rare', 'catbreed_siamese']
colors = ['has_Blue', 'has_Tortie',
       'has_White', 'has_Brown', 'has_Lilac', 'has_Point', 'has_Tabby',
       'has_Gray', 'has_Agouti', 'has_Black', 'has_Lynx', 'has_Orange',
       'has_Tan', 'has_Calico']

In [6]:
# rename column, fix this later
df_cat["catbreed"] = df_cat.hair_length
df_cat_dummies = pd.get_dummies(df_cat[cat_features + colors].dropna())

In [7]:
rows = np.random.choice(df_cat_dummies.index.values
                ,int( round(len(df_cat_dummies)*.666) ), replace=False)
df_cat_dummies_train = df_cat_dummies.ix[rows]
df_cat_dummies_test = df_cat_dummies.drop(rows)

#### fit and test model

In [18]:
train_features = df_cat_dummies_train[cat_features_dummies+colors]
train_outcomes = df_cat.ix[df_cat_dummies_train.index.values]['OutcomeType']
test_features  = df_cat_dummies_test[cat_features_dummies+colors]
test_outcomes  = df_cat.ix[df_cat_dummies_test.index.values]['OutcomeType']

In [19]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [21]:
bnb = BernoulliNB()
bnb.fit(train_features,train_outcomes)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [23]:
predictions = bnb.predict(test_features)
accuracy_score(test_outcomes,predictions)

0.729490022172949